# Testing single node datasets

Purpose: until now dataset represented by combination of tree types - now test for single node type

## Importing

In [1]:
import sys
import os

# Construct the path to the parent directory of the parent directory
parent_of_parent_dir = os.path.abspath(os.path.join(os.path.pardir, os.path.pardir))

# Add the parent directory of the parent directory to the system path
sys.path.insert(0, parent_of_parent_dir)

In [2]:
import optuna
import numpy as np
from sklearn.metrics import r2_score
import torch
from torch.utils.data import random_split, DataLoader

from etnn import TreeNode
from etnn.tools.training_tools import ConfigStore
from etnn.data.ferris_wheel import load_pure_ferris_wheel_dataset_single_node
from etnn.tools.training import train_epoch, eval_epoch
from etnn.nn.layer_framework import LayerManagementFramework
from etnn.tools.training_tools import seeding_all
from etnn.routines.run_config import choice_trainloader, choice_loss, choice_optim

## Control constants

In [3]:
dataset_path = "../../datasets/"
test_perc = 0.3
val_perc = 0.21
stability_count = 5
label = "tree-advanced" # although irrelevant

## Defining parameter search

In [4]:
def objective(trial):
    # init default config
    config = ConfigStore(
       in_dim=15,
       hidden_dim=trial.suggest_int("hidden_dim", 16, 512, step=16),
       out_dim=1,
       k=trial.suggest_int("k", 1, 5),
       dataset=0,
       ds_size=10_000,
       num_gondolas=-1,
       num_part_pg=-1,
       loss_name='mse',
       optimizer_name='adam',
       num_max_epochs=30, # real: 100
       learning_rate=trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
       batch_size=1024,
       early_stop_tol=5,
       use_equal_batcher=trial.suggest_categorical("batcher", [True, False]),
       seed=420,
       label_type=label,
       final_label_factor=5/1000
    )
    # config = ConfigStore(
    #     in_dim=15,
    #     hidden_dim=16,
    #     out_dim=1,
    #     k=2,
    #     dataset=-1 if normalized else 0,
    #     ds_size=10_000,
    #     num_gondolas=-1,
    #     num_part_pg=-1,
    #     loss_name='mse',
    #     optimizer_name='adam',
    #     num_max_epochs=30, # real: 100
    #     learning_rate=0.002,
    #     batch_size=1024,
    #     early_stop_tol=5,
    #     use_equal_batcher=False,
    #     seed=420,
    #     label_type=label,
    #     final_label_factor=5/1000
    # )

    # loading dataset
    dataset, df_index = load_pure_ferris_wheel_dataset_single_node(
        node_type=node_type,
        num_elem=size_elem,
        num_to_generate=config.ds_size,
        dataset_path=dataset_path,
        final_label_factor=config.final_label_factor,
        normalize=True if config.dataset == -1 else 0
    )

    if normalized:
        dataset.post_normalize()

    # splitting off test dataset
    generator = torch.Generator().manual_seed(config.seed)
    train_ds, val_ds, _ = random_split(
        dataset,
        [1 - test_perc - val_perc, val_perc, test_perc],
        generator=generator
    )

    # loaders
    train_loader = choice_trainloader(config, df_index, train_ds)
    val_loader = DataLoader(val_ds, batch_size=4 * config.batch_size, shuffle=False)

    # define device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # set seed for reproducability
    seeding_all(config.seed)

    # define model
    model = LayerManagementFramework(
        in_dim=config.in_dim,
        tree=TreeNode(node_type, [TreeNode("E", size_elem)]),
        hidden_dim=config.hidden_dim,
        out_dim=config.out_dim,
        k=config.k
    ).to(device)

    # learning tools
    criterion = choice_loss(config)
    optimizer = choice_optim(config, model)

    # init score list
    score_list = []

    # train for specified number of epochs
    for epoch in range(config.num_max_epochs):
        _, _, _ = train_epoch(
            model,
            train_loader,
            optimizer,
            device,
            criterion
        )

        _, val_true_y, val_pred_y = eval_epoch(
            model,
            val_loader,
            device,
            criterion
        )

        # calc r2 score and append
        score = r2_score(y_true=val_true_y, y_pred=val_pred_y)
        score_list += [score]
        trial.report(score, epoch)

    # calculate objective
    # display(score_list)
    # idea: last x r2 scores (why not last one? for stability purposes)
    obj = np.array(score_list)[-stability_count:]
    return np.mean(obj)

## Parameter search

### S

In [5]:
node_type = "S"
size_elem = 10
n_trials = 50
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-20 21:45:05,388] A new study created in memory with name: Study node type S, normalized: True


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-12-20 21:46:25,548] Trial 0 finished with value: 0.08934944980044916 and parameters: {'hidden_dim': 16, 'k': 5, 'learning_rate': 0.0012546091565712432, 'batcher': False}. Best is trial 0 with value: 0.08934944980044916.
[I 2023-12-20 21:47:45,784] Trial 1 finished with value: 0.8934739388185354 and parameters: {'hidden_dim': 176, 'k': 3, 'learning_rate': 0.0018136648460122967, 'batcher': True}. Best is trial 1 with value: 0.8934739388185354.
[I 2023-12-20 21:49:09,267] Trial 2 finished with value: 0.8001011246718497 and parameters: {'hidden_dim': 464, 'k': 1, 'learning_rate': 0.000741675256430483, 'batcher': True}. Best is trial 1 with value: 0.8934739388185354.
[I 2023-12-20 21:50:32,849] Trial 3 finished with value: 0.7689758527238156 and parameters: {'hidden_dim': 224, 'k': 5, 'learning_rate': 0.0003582036616317171, 'batcher': True}. Best is trial 1 with value: 0.8934739388185354.
[I 2023-12-20 21:52:16,283] Trial 4 finished with value: 0.7970284087099441 and parameters: {'h

In [6]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_post-normalized-{normalized}.csv")

### Q

In [7]:
node_type = "Q"
size_elem = 10
n_trials = 50
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-20 23:08:53,013] A new study created in memory with name: Study node type Q, normalized: True


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-12-20 23:10:30,608] Trial 0 finished with value: -0.058720025390144495 and parameters: {'hidden_dim': 144, 'k': 5, 'learning_rate': 5.653257441659244e-05, 'batcher': False}. Best is trial 0 with value: -0.058720025390144495.
[I 2023-12-20 23:12:04,174] Trial 1 finished with value: 0.926738050885713 and parameters: {'hidden_dim': 464, 'k': 2, 'learning_rate': 0.002559273511363111, 'batcher': True}. Best is trial 1 with value: 0.926738050885713.
[I 2023-12-20 23:13:36,128] Trial 2 finished with value: 0.03294403578654035 and parameters: {'hidden_dim': 80, 'k': 4, 'learning_rate': 0.00019776369616638932, 'batcher': False}. Best is trial 1 with value: 0.926738050885713.
[I 2023-12-20 23:15:09,393] Trial 3 finished with value: 0.9226773292606932 and parameters: {'hidden_dim': 64, 'k': 3, 'learning_rate': 0.005590225444094862, 'batcher': False}. Best is trial 1 with value: 0.926738050885713.
[I 2023-12-20 23:16:44,443] Trial 4 finished with value: 0.4254004377461073 and parameters: {

In [8]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_post-normalized-{normalized}.csv")

### C

In [9]:
node_type = "C"
size_elem = 10
n_trials = 50
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-21 00:51:03,772] A new study created in memory with name: Study node type C, normalized: True


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-12-21 00:53:13,026] Trial 0 finished with value: 0.8612508180404213 and parameters: {'hidden_dim': 272, 'k': 4, 'learning_rate': 0.0003508829737558717, 'batcher': True}. Best is trial 0 with value: 0.8612508180404213.
[I 2023-12-21 00:55:20,016] Trial 1 finished with value: 0.9048524827713831 and parameters: {'hidden_dim': 416, 'k': 3, 'learning_rate': 0.0005115964004865398, 'batcher': True}. Best is trial 1 with value: 0.9048524827713831.
[I 2023-12-21 00:57:27,202] Trial 2 finished with value: -179.63784086390714 and parameters: {'hidden_dim': 80, 'k': 4, 'learning_rate': 1.1071281645793394e-05, 'batcher': True}. Best is trial 1 with value: 0.9048524827713831.
[I 2023-12-21 00:59:34,419] Trial 3 finished with value: 0.9212202873969977 and parameters: {'hidden_dim': 112, 'k': 3, 'learning_rate': 0.003365359067528299, 'batcher': True}. Best is trial 3 with value: 0.9212202873969977.
[I 2023-12-21 01:01:41,619] Trial 4 finished with value: -75.53338789198128 and parameters: {'hi

In [10]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_post-normalized-{normalized}.csv")

### P

In [11]:
node_type = "P"
size_elem = 10
n_trials = 50
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-21 02:38:30,588] A new study created in memory with name: Study node type P, normalized: True


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-12-21 02:41:00,903] Trial 0 finished with value: 0.12052853097762983 and parameters: {'hidden_dim': 128, 'k': 1, 'learning_rate': 0.00042853643373171654, 'batcher': False}. Best is trial 0 with value: 0.12052853097762983.
[I 2023-12-21 02:43:31,264] Trial 1 finished with value: -78.59231117511418 and parameters: {'hidden_dim': 464, 'k': 5, 'learning_rate': 2.776930843753622e-05, 'batcher': True}. Best is trial 0 with value: 0.12052853097762983.
[I 2023-12-21 02:46:01,437] Trial 2 finished with value: 0.31604956832886 and parameters: {'hidden_dim': 32, 'k': 4, 'learning_rate': 0.006636929740437197, 'batcher': False}. Best is trial 2 with value: 0.31604956832886.
[I 2023-12-21 02:48:32,153] Trial 3 finished with value: -185.822005484006 and parameters: {'hidden_dim': 48, 'k': 4, 'learning_rate': 0.00011464010746841958, 'batcher': False}. Best is trial 2 with value: 0.31604956832886.
[I 2023-12-21 02:51:02,606] Trial 4 finished with value: -38.242050330046325 and parameters: {'hid

In [12]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_post-normalized-{normalized}.csv")